# Installs

In [1]:
# %pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q


This may take a while

In [2]:
# !pip install torchsummaryx==1.1.0
# !pip install wandb --quiet
# !pip install python-Levenshtein -q
# !git clone --recursive https://github.com/parlance/ctcdecode.git
# !pip install wget -q
# %cd ctcdecode
# !pip install . -q
# %cd ..

# Imports

In [3]:
!pip install torchaudio


In [4]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torchnlp.nn import LockedDropout
import torchaudio

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [5]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"aaayush1","key":"9ad570c944c8b67cf242fb16c7cf3f40"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
# !kaggle competitions download -c hw3p2-785-f24

In [7]:
# '''
# This will take a couple minutes, but you should see at least the following:
# 11-785-f24-hw3p2  ctcdecode  hw3p2asr-f24.zip  sample_data
# '''
# !unzip -q hw3p2-785-f24.zip
# !ls

# Google Drive

In [8]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [9]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]
# PHONEMES = sorted(PHONEMES)
# LABELS = sorted(LABELS)



In [10]:
# You might want to play around with the mapping as a sanity check here
print(PHONEMES)
print(len(PHONEMES))
print(LABELS)
print(len(LABELS))

['', '[SIL]', 'NG', 'F', 'M', 'AE', 'R', 'UW', 'N', 'IY', 'AW', 'V', 'UH', 'OW', 'AA', 'ER', 'HH', 'Z', 'K', 'CH', 'W', 'EY', 'ZH', 'T', 'EH', 'Y', 'AH', 'B', 'P', 'TH', 'DH', 'AO', 'G', 'L', 'JH', 'OY', 'SH', 'D', 'AY', 'S', 'IH']
41
[' ', '-', 'G', 'f', 'm', '@', 'r', 'u', 'n', 'i', 'W', 'v', 'U', 'o', 'a', 'R', 'h', 'z', 'k', 'C', 'w', 'e', 'Z', 't', 'E', 'y', 'A', 'b', 'p', 'T', 'D', 'c', 'g', 'l', 'j', 'O', 'S', 'd', 'Y', 's', 'I']
41


In [11]:
mfcc_dir = os.path.join('/content/11785-f24-hw3p2', 'train-clean-100', "mfcc")
# TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root

# TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
mfcc_names          = sorted(os.listdir(mfcc_dir))
transcript_dir = os.path.join("/content/11785-f24-hw3p2", "train-clean-100", "transcript")

mfcc_files = sorted(os.listdir(mfcc_dir))
transcript_files = sorted(os.listdir(transcript_dir))



# TODO: Iterate through mfccs and transcripts
mfcc_lengths = []

# Iterate through MFCC files
for i in range(len(mfcc_names)):
    mfcc_path = os.path.join(mfcc_dir, mfcc_names[i])

    # Load the MFCC
    mfcc = np.load(mfcc_path)

    # Append the number of time steps (shape[0]) to the list
    mfcc_lengths.append(mfcc.shape[0])

    transcript_path = os.path.join(transcript_dir,transcript_files[i])
    transcript  = np.load(transcript_path)

    print(mfcc.shape)
    print(transcript.shape)

    if i == 3:
      break

    # print(transcript)




# Calculate the mean, min, and max
# mean_length = np.mean(mfcc_lengths)
# min_length = np.min(mfcc_lengths)
# max_length = np.max(mfcc_lengths)

# print(mean_length)
# print(min_length)
# print(max_length)



(1404, 28)
(147,)
(1590, 28)
(196,)
(1390, 28)
(179,)
(1467, 28)
(183,)


### Train Data

In [12]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, partition="train-clean-100"):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(PHONEMES)}
        index_to_phoneme = {idx: phoneme for idx, phoneme in enumerate(PHONEMES)}

        # Now assign them to the class attributes
        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        self.phoneme_to_index = phoneme_to_index
        self.index_to_phoneme = index_to_phoneme


        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''

        self.mfccs, self.transcripts = [], []

        for i in range(self.length):
            mfcc_path = os.path.join(self.mfcc_dir, self.mfcc_files[i])
            transcript_path = os.path.join(self.transcript_dir, self.transcript_files[i])
            mfcc        = np.load(mfcc_path)

            mfcc = mfcc - np.mean(mfcc, axis=0, keepdims=True)
            mfcc = mfcc / np.std(mfcc, axis=0, keepdims=True)

            transcript  = np.load(transcript_path)[1:-1]
            self.mfccs.append(mfcc)
            self.transcripts.append([self.phoneme_to_index[i] for i in transcript])


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind]

        transcript = self.transcripts[ind]
        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [item[0] for item in batch] # TODO
        # batch of output phonemes
        batch_transcript = [torch.tensor(item[1], dtype=torch.long) for item in batch]

        lengths_mfcc = [mfcc.shape[0] for mfcc in batch_mfcc]
        lengths_transcript = [len(transcript) for transcript in batch_transcript]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(
                [torch.tensor(mfcc, dtype=torch.float32) for mfcc in batch_mfcc],
                batch_first=True,
                padding_value=0.0
            )

            # Use a custom padding value (-1) for the transcripts
        batch_transcript_pad = pad_sequence(
                batch_transcript,
                batch_first=True,
                padding_value=-1
            )
        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [13]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):

  def __init__(self, root, partition="test-clean"):
    '''
    Initializes the test dataset.

    Args:
        root (str): Root directory containing the dataset.
        partition (str): Partition of the dataset (e.g., "test-clean").
    '''

    # Load the directory for MFCC files
    self.mfcc_dir = os.path.join(root, partition, "mfcc")
    self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

    # Length of the dataset
    self.length = len(self.mfcc_files)

    # Load and normalize all MFCC features
    self.mfccs = []
    for i in range(self.length):
        mfcc_path = os.path.join(self.mfcc_dir, self.mfcc_files[i])
        mfcc = np.load(mfcc_path)

        # Normalization
        mfcc = (mfcc - np.mean(mfcc, axis=0, keepdims=True)) / np.std(mfcc, axis=0, keepdims=True)
        self.mfccs.append(mfcc)

  def __len__(self):
    ''' Returns the total number of items in the dataset. '''
    return self.length

  def __getitem__(self, ind):
    '''
    Returns the MFCC coefficients for the given index.

    Args:
        ind (int): Index of the item.

    Returns:
        numpy array: MFCC features.
    '''
    return self.mfccs[ind]

  def collate_fn(self, batch):
    '''
    Collate function for batching test data.

    Args:
        batch: List of MFCC features.

    Returns:
        tuple: Padded mfcc features and lengths of original features.
    '''

    # Extract features and calculate lengths
    batch_mfcc = [torch.tensor(item, dtype=torch.float32) for item in batch]
    lengths_mfcc = [mfcc.shape[0] for mfcc in batch_mfcc]

    # Pad the mfcc features
    batch_mfcc_pad = pad_sequence(
        batch_mfcc,
        batch_first=True,
        padding_value=0.0  # Default padding for MFCCs (silent frame)
    )

    # Return padded features and lengths
    return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Config - Hyperparameters

In [14]:
root = "/content/11785-f24-hw3p2"

# Feel free to add more items here
config = {
    "beam_width" : 2,
    "lr"         : 2e-3,
    "epochs"     : 60,
    "batch_size" : 64  # Increase if your device can handle it
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [15]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [16]:
# Create objects for the dataset class
# root, partition="test-clean"):
# /content/11785-f24-hw3p2/train-clean-100

train_data = AudioDataset(root, partition="train-clean-100") #TODO
val_data = AudioDataset(root, partition="dev-clean") # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(root, partition="test-clean") #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=config["batch_size"],  # You can set this based on your preference or system capabilities
    shuffle=True,
    collate_fn=train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=config["batch_size"],  # You can set this based on your preference or system capabilities
    shuffle=False,
    collate_fn=val_data.collate_fn
)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=config["batch_size"],  # You can set this based on your preference or system capabilities
    shuffle=False,
    collate_fn=test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))


Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [17]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1601, 28]) torch.Size([64, 195]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [18]:
# torch.cuda.empty_cache()

# class Network(nn.Module):

#     def __init__(self, input_size, out_size):

#         super(Network, self).__init__()

#         # Adding some sort of embedding layer or feature extractor might help performance.
#         # self.embedding = ?

#         # TODO : look up the documentation. You might need to pass some additional parameters.
#         self.lstm = nn.LSTM(input_size=input_size, hidden_size=256, num_layers=1, batch_first=True)

#         self.classification = nn.Linear(in_features=256, out_features=out_size)


#         self.logSoftmax = nn.LogSoftmax(dim=-1)

#     def forward(self, x, lx):
#         #TODO
#         # The forward function takes 2 parameter inputs here. Why?
#         # Refer to the handout for hints
#         packed_x = nn.utils.rnn.pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)

#         # Process with LSTM
#         # packed_output, _ = self.lstm(packed_x)
#         output, _ = self.lstm(x)

#         # Unpack the sequence after LSTM
#         # output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

#         # Classification and log softmax
#         output = self.classification(output)
#         output = self.logSoftmax(output)

#         return output


## Initialize Basic Network
(If trying out the basic Network)

In [19]:
# torch.cuda.empty_cache()

# model = Network(input_size=28, out_size=len(PHONEMES)).to(device)
# summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

## ASR Network

**ResNet for Embeddings**

In [20]:
# class IdentityBlock1D(torch.nn.Module):
#     def __init__(self, in_channels, filters):
#         super(IdentityBlock1D, self).__init__()
#         f1, f2 = filters

#         self.conv1 = torch.nn.Conv1d(in_channels, f1, kernel_size=3, stride=1, padding=1)
#         self.bn1 = torch.nn.BatchNorm1d(f1)

#         self.conv2 = torch.nn.Conv1d(f1, f2, kernel_size=3, stride=1, padding=1)
#         self.bn2 = torch.nn.BatchNorm1d(f2)

#         self.relu = torch.nn.ReLU()

#         self.adjust_channels = None
#         if in_channels != f2:
#             self.adjust_channels = torch.nn.Conv1d(in_channels, f2, kernel_size=1, stride=1)

#     def forward(self, x):
#         identity = x

#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)

#         out = self.conv2(out)
#         out = self.bn2(out)

#         # Adjust the channels of `identity` if needed
#         if self.adjust_channels is not None:
#             identity = self.adjust_channels(identity)

#         out += identity
#         out = self.relu(out)

#         return out

# class ResNet18(torch.nn.Module):
#     def __init__(self, input_dim, embedding_dim):
#         super(ResNet18, self).__init__()

#         self.conv1 = torch.nn.Conv1d(input_dim, 64, kernel_size=7, stride=1, padding=3)  # Initial layer
#         self.bn1 = torch.nn.BatchNorm1d(64)
#         self.relu = torch.nn.ReLU()

#         # Layer 1
#         self.layer1 = torch.nn.Sequential(
#             IdentityBlock1D(64, [64, 64]),
#             IdentityBlock1D(64, [64, 64])
#         )

#         # Layer 2
#         self.layer2 = torch.nn.Sequential(
#             IdentityBlock1D(64, [128, 128]),
#             IdentityBlock1D(128, [128, 128])
#         )

#         # Layer 3
#         self.layer3 = torch.nn.Sequential(
#             IdentityBlock1D(128, [256, 256]),
#             IdentityBlock1D(256, [256, 256])
#         )

#         # Layer 4
#         self.layer4 = torch.nn.Sequential(
#             IdentityBlock1D(256, [embedding_dim, embedding_dim]),
#             IdentityBlock1D(embedding_dim, [embedding_dim, embedding_dim])
#         )

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)

#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)

#         return x


# SeNet for Embeddings


In [21]:
class SEBlock1D(torch.nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock1D, self).__init__()
        self.global_avg_pool = torch.nn.AdaptiveAvgPool1d(1)
        self.fc1 = torch.nn.Linear(channels, channels // reduction, bias=False)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        batch, channels, _ = x.size()
        # Squeeze
        y = self.global_avg_pool(x).view(batch, channels)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(batch, channels, 1)
        # Excitation
        return x * y  # Scale the input features

class IdentityBlock1D(torch.nn.Module):
    def __init__(self, in_channels, filters):
        super(IdentityBlock1D, self).__init__()
        f1, f2 = filters

        self.conv1 = torch.nn.Conv1d(in_channels, f1, kernel_size=3, stride=1, padding=1)
        self.bn1 = torch.nn.BatchNorm1d(f1)

        self.conv2 = torch.nn.Conv1d(f1, f2, kernel_size=3, stride=1, padding=1)
        self.bn2 = torch.nn.BatchNorm1d(f2)

        self.se_block = SEBlock1D(f2)  # Add SE Block after the second convolution
        self.relu = torch.nn.ReLU()

        self.adjust_channels = None
        if in_channels != f2:
            self.adjust_channels = torch.nn.Conv1d(in_channels, f2, kernel_size=1, stride=1)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Apply SE block
        out = self.se_block(out)

        # Adjust the channels of `identity` if needed
        if self.adjust_channels is not None:
            identity = self.adjust_channels(identity)

        out += identity
        out = self.relu(out)

        return out

class ResNet18(torch.nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(ResNet18, self).__init__()

        self.conv1 = torch.nn.Conv1d(input_dim, 64, kernel_size=7, stride=1, padding=3)  # Initial layer
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.relu = torch.nn.ReLU()

        # Layer 1
        self.layer1 = torch.nn.Sequential(
            IdentityBlock1D(64, [64, 64]),
            IdentityBlock1D(64, [64, 64])
        )

        # Layer 2
        self.layer2 = torch.nn.Sequential(
            IdentityBlock1D(64, [128, 128]),
            IdentityBlock1D(128, [128, 128])
        )

        # Layer 3
        self.layer3 = torch.nn.Sequential(
            IdentityBlock1D(128, [256, 256]),
            IdentityBlock1D(256, [256, 256])
        )

        # Layer 4
        self.layer4 = torch.nn.Sequential(
            IdentityBlock1D(256, [embedding_dim, embedding_dim]),
            IdentityBlock1D(embedding_dim, [embedding_dim, embedding_dim])
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        return x


### Pyramid Bi-LSTM (pBLSTM)

In [22]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [23]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size=input_size,
                             hidden_size=hidden_size,
                             num_layers=1,
                             bidirectional=True,
                             batch_first=True)



    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        # TODO: Pass the sequence through bLSTM

        # What do you return?

        x_packed, _ = self.blstm(x_packed)

        x, x_lens = nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)

        x, x_lens = self.trunc_reshape(x, x_lens)

        # Step 5: Re-pack the downsampled sequence for the next layer (if any)
        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)

        # Return the final packed output
        return x_packed

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor
        if x.size(1) % 2 != 0:
            x = x[:, :-1, :]

        # Reshape by concatenating every two time steps
        batch_size, seq_len, input_size = x.size()

        x = x.view(batch_size, seq_len // 2, input_size * 3)  # New shape: (batch_size, seq_len // 2, input_size * 2)

        # Update sequence lengths by dividing by 2
        x_lens = x_lens // 2

        return x, x_lens

### Encoder

In [24]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()


        self.embedding = torch.nn.Sequential(
            PermuteBlock(),
            ResNet18(input_dim=input_size, embedding_dim=encoder_hidden_size),
            PermuteBlock()
        )

        self.pBLSTMs = nn.Sequential(
            pBLSTM(input_size=encoder_hidden_size, hidden_size=encoder_hidden_size),
            pBLSTM(input_size=encoder_hidden_size * 4, hidden_size=encoder_hidden_size)
        )

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence
        x = self.embedding(x)

        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)

        x_packed = self.pBLSTMs(x_packed)

        encoder_outputs, encoder_lens = nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)

        return encoder_outputs, encoder_lens

### Decoder

In [25]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(),
            torch.nn.BatchNorm1d(embed_size),
            PermuteBlock(),
            torch.nn.Linear(embed_size, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, output_size)
        )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification
        out = self.mlp(encoder_out)

        # Apply LogSoftmax for classification probabilities
        out = self.softmax(out)
        return out

In [26]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 512, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations = torch.nn.Sequential(
            PermuteBlock(),
            torchaudio.transforms.FrequencyMasking(freq_mask_param=3),
            torchaudio.transforms.TimeMasking(time_mask_param=5),
            PermuteBlock()  # Return to original dimension order
        )
        self.encoder = Encoder(input_size=input_size, encoder_hidden_size=embed_size)
        self.decoder = Decoder(embed_size=embed_size * 4, output_size=output_size)



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [27]:
print(x.shape, lx.shape)

torch.Size([64, 1601, 28]) torch.Size([64])


In [28]:
from torchinfo import summary
model = ASRModel(
    input_size  = 28,
    embed_size  = 512,
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, input_data=x.to(device), lengths_x=lx)

ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): FrequencyMasking()
    (2): TimeMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): ResNet18(
        (conv1): Conv1d(28, 64, kernel_size=(7,), stride=(1,), padding=(3,))
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (layer1): Sequential(
          (0): IdentityBlock1D(
            (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
            (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
            (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (se_block): SEBlock1D(
              (global_avg_pool): AdaptiveAvgPool1d(output_size=1)
              (fc1): Linear(in_features=

Layer (type:depth-idx)                                       Output Shape              Param #
ASRModel                                                     [64, 400, 41]             --
├─Encoder: 1-1                                               [64, 400, 2048]           --
│    └─Sequential: 2-1                                       [64, 1601, 512]           --
│    │    └─PermuteBlock: 3-1                                [64, 28, 1601]            --
│    │    └─ResNet18: 3-2                                    [64, 512, 1601]           3,946,048
│    │    └─PermuteBlock: 3-3                                [64, 1601, 512]           --
│    └─Sequential: 2-2                                       [18269, 2048]             --
│    │    └─pBLSTM: 3-4                                      [36572, 2048]             4,202,496
│    │    └─pBLSTM: 3-5                                      [18269, 2048]             10,493,952
├─Decoder: 1-2                                               [64, 400, 41

# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [29]:
#TODO


criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=1e-3)

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(
    labels=PHONEMES,
    blank_id=0,
    beam_width=config["beam_width"],          # Number of beams to consider
    num_processes=4,         # Number of parallel processes to speed up decoding
    log_probs_input=True     # Whether the input to the decoder is in log probability form
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=4,  # Initial restart period
    T_mult=2,     # Multiplicative factor for restart period
    eta_min=1e-5
)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [30]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, _, _, out_lens = decoder.decode(output, seq_lens=output_lens) #lengths - list of lengths

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred_indices = beam_results[i][0][:out_lens[i][0]]
        pred_string = ''.join([PHONEME_MAP[idx] for idx in pred_indices])
        pred_strings.append(pred_string)

    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string =  pred_strings[i]
        label_indices = label[i][:label_lens[i]]
        label_string = ''.join([PHONEME_MAP[idx] for idx in label_indices])
        print("Pred String is, ", pred_string, "Label String is:" ,label_string)
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist
    # return dist

# Test Implementation

In [31]:
# test code to check shapes
torch.cuda.empty_cache()

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h_permuted = torch.permute(h, (1, 0, 2))
    print(h_permuted.shape, y.shape)
    loss = criterion(h_permuted, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(7.6832, device='cuda:0', grad_fn=<MeanBackward0>)
Pred String is,  DODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODO Label String is: -mIstRkwIltRIzDApasAlAvDAmIdAlkl@sAzAndwiargl@dtuwElkAmhIzgaspAl-
Pred String is,  DODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODOD Label String is: -ncrIzmIstRkwIltRzm@nRlEsIntrAstIGD@nhIzm@tR-
Pred String is,  DODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODODOD

# WandB

You will need to fetch your api key from wandb.ai

In [32]:
import wandb
wandb.login(key="7664f3b17a98ffe7c64b549e349123b61a9d3024")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aaayush (aaayush-carnegie-mellon-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [33]:
run = wandb.init(
    name = "Better-Performance1-1", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

# Train Functions

In [34]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [35]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [36]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/content/checkpoint/last_model.pth"
best_model_path = "/content/checkpoint/best_model.pth"

In [37]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])
    train_loss = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist = validate_model(model, val_loader, decoder)

    scheduler.step()

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/60


	Train Loss 2.0181	 Learning Rate 0.0020000
	Val Dist 23.0059	 Val Loss 1.0384


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved epoch model
Saved best model

Epoch: 2/60


	Train Loss 0.7505	 Learning Rate 0.0017086
	Val Dist 15.0506	 Val Loss 0.6881
Saved epoch model
Saved best model

Epoch: 3/60


...

	Train Loss 0.0272	 Learning Rate 0.0010050
	Val Dist 6.1048	 Val Loss 0.5492
Saved epoch model

Epoch: 46/60


	Train Loss 0.0230	 Learning Rate 0.0009075
	Val Dist 6.1473	 Val Loss 0.5245
Saved epoch model

Epoch: 47/60


	Train Loss 0.0208	 Learning Rate 0.0008109
	Val Dist 6.0611	 Val Loss 0.5608
Saved epoch model

Epoch: 48/60


Train:  14%|█▍        | 64/446 [01:33<09:28,  1.49s/it, loss=0.0161, lr=0.000716]

KeyboardInterrupt: 

# Generate Predictions and Submit to Kaggle

In [38]:
checkpoint = torch.load("/content/checkpoint/best_model.pth")

model.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [39]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 5

test_decoder = CTCBeamDecoder(
    labels=LABELS,
    blank_id=0,
    beam_width=TEST_BEAM_WIDTH,
    num_processes=4,
    log_probs_input=True
)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_strings = decode_prediction(h, lh, test_decoder)
    #TODO save the output in results array.

    results.extend(prediction_strings)
    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing



100%|██████████| 41/41 [00:46<00:00,  1.14s/it]


In [40]:
df = pd.DataFrame({"index": range(len(results)), "label": results})

# Save to the specified file path
df.to_csv("/content/submission.csv", index=False)

In [41]:
!kaggle competitions submit -c 11-785-hw3p2-f24 -f /content/submission.csv -m "Better-Performance"


100% 209k/209k [00:00<00:00, 351kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)